# Фабарисов Дмитрий. Выпуклый анализ и оптимизация. Задание 1.

In [159]:
import cvxopt as cp
cp.solvers.options['show_progress'] = False

from scipy import optimize as opt
import copy as cp
import numpy as np
import matplotlib.pyplot as plt


%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Создадим вспомогательные функции

$$f'' (x) =
\begin{pmatrix}
48 x_1^2 - 8 x_2 - 7.5 & -8 x_1 & 0 & ... & 0 & 0\\
-8 x_1 & 48 x_2^2 - 8 x_3 - 6 & -8 x_2 & ... & 0 & 0\\
0 & -8 x_2 & 48 x_3^2 - 8 x_4 - 6 & ... & 0 & 0\\
... & & & & & \\
0 & 0 & 0 & ... & 48 x_{n-1}^2 - 8 x_n - 6 & -8 x_{n-1} \\
0 & 0 & 0 & ... & -8 x_{n-1} & 2 \\
\end{pmatrix}$$

In [160]:
def check_difference(func, grad_func, f_true, x, x_true):
    delta_x = np.linalg.norm(x - x_true)
    delta_func = np.abs(func(x) - f_true)
    abs_grad = np.linalg.norm(grad_func(x))
    
    return delta_x, delta_func, abs_grad 

In [161]:
def func(x):
    l = len(x)
    res = 0
    res += (x[0] - 1)**2 / 4
    for i in range(1, l):
        res += (2 * x[i-1]**2 - x[i] - 1) ** 2
    return res

In [162]:
def grad_func(x):
    l = len(x)
    grad = np.zeros(l)
    grad[0] = 16*x[0]**3 - 8*x[0]*x[1] - 7.5*x[0] - 0.5
    grad[l-1] = -4*x[l-2]**2 + 2*x[l-1] + 2
    for i in range(1, l-1):
        grad[i] = 16*x[i]**3 - 4*x[i-1]**2 - 8*x[i]*x[i+1] - 6*x[i] + 2
    
    return grad

In [163]:
def hess_func(x):
    n = len(x)
    hess = np.zeros([n, n])
    hess[0,0] = 48*x[0]**2 - 8*x[1] - 7.5
    hess[0,1] = -8*x[0]
    hess[1,0] = hess[0,1]
    hess[n-1, n-1] = 2
    hess[n-1, n-2] = -8*x[n-2]
    hess[n-2, n-1] = hess[n-1, n-2]
    for i in range(1, n-1):
        hess[i, i] = 48*x[i]**2 - 8*x[i+1] - 6
        
        hess[i, i-1] = -8*x[i-1]
        hess[i, i+1] = -8*x[i+1]
        
        hess[i-1, i] = hess[i, i-1]
        hess[i+1, i] = hess[i, i+1]
    
    return np.matrix(hess)

## Задаем размерность, генерируем $x$, задаем истинные значения минимума $x_{true}$, $y_{true}$.

In [193]:
dimension = 3
x = [1.5] + [1.0]*(dimension - 1)

x_true = [1.0] * dimension
f_true = 0.0

## Метод градиентного спуска

In [166]:
def gradient_descent(x0, func, grad_func, check_difference, step, tol, max_steps=100000):
    x = np.array(x0)
    i = 0
    while True:

        stats = check_difference(func, grad_func, f_true, x, x_true)
        if (i > max_steps):
            return stats
        if (stats[2] < tol):
            return x, i, stats[2], stats[0], stats[1] 
        
        x -= step * grad_func(x)
        i += 1

In [167]:
#print gradient_descent(x, func, grad_func, check_difference, 0.01, 10 ** (-6))

## Метод сопряженных градиентов

In [168]:
def conjugate_gradient(x0, func, grad_func, check_difference, tol, max_steps=10000):
    x = np.array(x0)
    i = 0
    S = -grad_func(x)
    new_grad = -S
    while True:
        i += 1
        stats = check_difference(func, grad_func, f_true, x, x_true)
        
        if (i > max_steps):
            return "too much steps", x, i, stats
        if (stats[2] < tol):
            return x, i, stats[2], stats[0], stats[1] 
        def func_lambd(lambd):
            return func(x + lambd * S)
        prev_grad = new_grad
        lambd = opt.minimize(func_lambd, 0.0, method='Powell').x
        x += lambd*S
        new_grad = grad_func(x)
        w = np.linalg.norm(new_grad)**2 / np.linalg.norm(prev_grad)**2
        S = -new_grad + w*S
         

In [186]:
#print conjugate_gradient(x, func, grad_func, check_difference, 10 ** (-9), 100)

## Метод наискорейшего спуска

In [170]:
def optimal_gradient_descent(x0, func, grad_func, check_difference, tol, max_steps=10000):
    x = np.array(x0)
    i = 0
    while True:
        i += 1
        stats = check_difference(func, grad_func, f_true, x, x_true)
        if (i > max_steps):
            return "too much steps", x, i, stats
        if (stats[2] < tol):
            return x, i, stats[2], stats[0], stats[1]
        
        S = -grad_func(x)
        def func_lambd(lambd):
            return func(x + lambd * S)
        lambd = opt.minimize(func_lambd, 0.0, method='Powell').x
        x += lambd*S
        

In [185]:
#print optimal_gradient_descent(x, func, grad_func, check_difference, 10 ** (-8), 1000)

In [191]:
def newton_method(x0, func, grad_func, hess_func, check_difference, tol, max_steps=100):
    x = np.array(x0)
    i = 0
    while True:
        i += 1
        stats = check_difference(func, grad_func, f_true, x, x_true)
        if (i > max_steps):
            return "too much steps", x, i, stats
        if (stats[2] < tol):
            return x, i, stats[2], stats[0], stats[1]
        micro_step = 0.001
        S = grad_func(x)
        H = hess_func(x)
        res = H.I * np.matrix(S).T
        x -= np.asarray(res).ravel()
        if (i % 2000 == 0):
            print i, x, func(x), numpy.linalg.det(H)

In [192]:
print newton_method(x, func, grad_func, hess_func, check_difference, 10 ** (-2), 100000)

2000 [  1.40756317   2.98056518  16.76829658] 0.0418550128114 -1111006.95438
4000 [  1.41236245   3.00778326  17.09427859] 0.0428442576095 -1163208.95635
6000 [  1.4170423    3.03441179  17.4160676 ] 0.0438199846534 -1216120.26958
8000 [  1.421609     3.06048004  17.73383317] 0.0447826839912 -1269726.86242
10000 [  1.42606833   3.086015    18.0477335 ] 0.0457328172357 -1324015.36506
12000 [  1.43042563   3.11104169  18.3579165 ] 0.046670819786 -1378973.02101
14000 [  1.43468583   3.13558334  18.66452068] 0.0475971028346 -1434587.6434
16000 [  1.43885352   3.15966152  18.96767603] 0.0485120551863 -1490847.57545
18000 [  1.44293293   3.18329635  19.26750467] 0.0494160449071 -1547741.65456
20000 [  1.44692805   3.20650659  19.56412158] 0.050309420823 -1605259.17969
22000 [  1.45084255   3.22930978  19.85763512] 0.0511925138851 -1663389.88166
24000 [  1.45467989   3.25172236  20.14814758] 0.0520656384132 -1722123.89596
26000 [  1.45844329   3.27375973  20.43575565] 0.0529290932323 -1781451